In [8]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta


In [32]:
# Download data function
def download_data(ticker_symbols, start_date,  end_date):
        data = {}
        for symbol in ticker_symbols:
                symbol_data = yf.download(symbol, start=start_date, end=end_date)
                data[symbol] = symbol_data['Close']
        symbol_prices = pd.DataFrame(data)
        return symbol_prices

In [39]:
def calculate_rsi(ticker_data):
    delta = ticker_data.diff().dropna()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    # rolling window of 14 days is recommended
    avg_gain = gain.rolling(window=14, min_periods=1).mean()
    avg_loss = loss.rolling(window=14, min_periods=1).mean()
    rsi = 100 - (100/(1 + (avg_gain/avg_loss)))
    return rsi


In [63]:
def analyze_prices(df):
    for ticker in df.columns:
        # Calculate RSI for each stock's closing prices
        rsi_series = calculate_rsi(df[ticker])

        buy_threshold=30 
        sell_threshold=70
        signals = []
        # Iterate through each day to check for signals
        for i in range(1, len(rsi_series)):
            previous_rsi = rsi_series.iloc[i - 1]
            current_rsi = rsi_series.iloc[i]
            date = rsi_series.index[i]

            # Check if a buy or sell signal is triggered
            if previous_rsi >= buy_threshold and current_rsi < buy_threshold:
                signal = "Buy"
                # print(f"Buy signal triggered for {ticker} on {date.strftime('%Y-%m-%d')} (Previous RSI: {previous_rsi:.2f}, Current RSI: {current_rsi:.2f})")
                signals.append([date, ticker, signal, current_rsi])
            elif previous_rsi <= sell_threshold and current_rsi > sell_threshold:
                signal = "Sell"
                # print(f"Sell signal triggered for {ticker} on {date.strftime('%Y-%m-%d')} (Previous RSI: {previous_rsi:.2f}, Current RSI: {current_rsi:.2f})")
                signals.append([date, ticker, signal, current_rsi])

    # Convert the list of signals into a DataFrame
    signals_df = pd.DataFrame(signals, columns=["Date", "Ticker", "Signal", "RSI"])
    return signals_df

In [64]:
def main():
    # 5 most active stocks from yahoo finance
    stocks = ['NVDA', 'NIO', 'TSLA', 'INND', 'INTC']
    # calculate the today's date and the date exactly 1 years in the past
    present_date = date.today()
    past_date = present_date - relativedelta(years=1)
    present_date = present_date.strftime('%Y-%m-%d')
    past_date = past_date.strftime('%Y-%m-%d')

    # download the etf data from dates above
    stock_data = download_data(stocks, past_date, present_date)
    signal_df = analyze_prices(stock_data)
    print(signal_df)

    # Save the signals DataFrame to a CSV file
    signal_df.to_csv("rsi_signals.csv", index=False)
    print("\nSignals saved to rsi_signals.csv")
if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

         Date Ticker Signal        RSI
0  2023-09-11   INTC   Sell  77.333184
1  2023-09-19   INTC    Buy  25.531952
2  2023-09-29   INTC    Buy  29.624656
3  2023-10-10   INTC   Sell  70.714340
4  2023-10-16   INTC   Sell  76.923094
5  2023-10-26   INTC    Buy  27.148202
6  2023-11-10   INTC   Sell  71.839589
7  2023-11-14   INTC   Sell  80.519464
8  2023-12-22   INTC   Sell  74.458875
9  2024-01-17   INTC    Buy  23.758859
10 2024-02-08   INTC    Buy  22.112522
11 2024-04-08   INTC    Buy  28.789235
12 2024-04-10   INTC    Buy  28.849407
13 2024-04-12   INTC    Buy  25.108541
14 2024-07-08   INTC   Sell  79.698018
15 2024-07-12   INTC   Sell  71.683693
16 2024-07-29   INTC    Buy  28.240733
17 2024-08-01   INTC    Buy  19.777764

Signals saved to rsi_signals.csv
